Imports & Client Setup

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


Helper Function

In [11]:
def generate(prompt, temperature=0.5, max_tokens=200):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1
    )
    return response.choices[0].message.content


SECTION 1 — ZERO-SHOT PROMPTING

## Zero-Shot Prompt

We provide *only* instructions, no examples.

### Review:
“The cupcakes tasted really good but the frosting was too sweet, and delivery arrived 30 minutes late.”


In [12]:
review_1 = """
The cupcakes tasted really good but the frosting was too sweet, 
and delivery arrived 30 minutes late.
"""

zero_shot_prompt = f"""
Summarize the following bakery review in 3 bullet points, focusing on taste and delivery:

Review: "{review_1}"
"""

zero_shot_output = generate(zero_shot_prompt)
print(zero_shot_output)


- The cupcakes had a great taste overall.
- The frosting was excessively sweet.
- Delivery was delayed by 30 minutes.


SECTION 2 — FEW-SHOT PROMPTING

## Few-Shot Prompt

We give the model multiple examples so it learns:
- tone  
- structure  
- summary format  


In [13]:
review_2 = """
The custom birthday cake looked beautiful, but the sponge was dry and lacked flavor. Not worth the price.
"""

few_shot_prompt = f"""
You are a bakery customer-experience analyst.
Summaries must always follow this format:

- Good aspects
- Problems
- Additional notes (if any)

Example 1:
Review: "The cupcakes tasted really good but the frosting was too sweet, and delivery arrived 30 minutes late."
Summary:
- Great cupcake flavor
- Frosting too sweet; delivery late
- Customer satisfied with taste overall

Example 2:
Review: "Absolutely loved the chocolate tart! Fresh, rich, and perfectly balanced."
Summary:
- Excellent freshness and rich flavor
- No issues reported
- Customer would order again

Now summarize this review using the same 3-line format:

Review: "{review_2}"
"""

few_shot_output = generate(few_shot_prompt)
print(few_shot_output)


Summary:
- Beautiful appearance of the cake
- Dry sponge with lack of flavor; not worth the price
- Customer disappointed with taste and value


SECTION 3 — CHAIN-OF-THOUGHT PROMPTING

## Chain-of-Thought Prompt

We ask the model to think step-by-step before giving the final summary.

Review:
“The custom birthday cake looked beautiful, but the sponge was dry and lacked flavor. Not worth the price.”


In [14]:
cot_prompt = f"""
Analyze this bakery review step by step. Think through each stage carefully.

1. Identify positive elements.
2. Identify negative elements.
3. Identify emotional tone.
4. Give a final quality score from 1 to 5.
5. Provide a final short summary.

Review: "{review_2}"

Explain your reasoning step-by-step, then give the final answer.
"""

cot_output = generate(cot_prompt)
print(cot_output)


Sure! Let's analyze the bakery review step by step.

### 1. Identify Positive Elements
- **Visual Appeal**: The review states that "The custom birthday cake looked beautiful." This indicates that the cake had a pleasing appearance, which is a positive element.

### 2. Identify Negative Elements
- **Texture and Taste**: The reviewer mentions that "the sponge was dry and lacked flavor." This indicates dissatisfaction with the cake's texture and taste.
- **Value for Money**: The phrase "Not worth the price" suggests that the reviewer felt the cake did not provide good value for the money spent, which is a negative element.

### 3. Identify Emotional Tone
- The overall tone of the review is somewhat disappointed and critical. The reviewer appreciates the visual aspect of the cake but expresses dissatisfaction with its taste and value. The use of "not worth the price" conveys a sense of disappointment and frustration.

### 4. Give a Final Quality Score from 1 to 


SECTION 4 — COMPARISON TABLE

In [15]:
df_compare = pd.DataFrame({
    "Prompt Type": ["Zero-Shot", "Few-Shot", "Chain-of-Thought"],
    "Output": [zero_shot_output, few_shot_output, cot_output]
})

df_compare


,Prompt Type,Output
0,Zero-Shot,- The cupcakes had a great taste overall.\n- T...
1,Few-Shot,Summary:\n- Beautiful appearance of the cake\n...
2,Chain-of-Thought,Sure! Let's analyze the bakery review step by ...


# Observations

- Zero-shot produced a general summary but lacked consistency.
- Few-shot produced structured, predictable outputs matching the examples.
- Chain-of-thought gave the deepest analysis with step-by-step reasoning.
- Few-shot is best for consistent summaries.
- CoT is best for insight and customer sentiment analysis.
